# Importing Python Libraries

In [2]:
# Installing libraries
!pip install jupyter_bokeh
!pip install hvplot

In [3]:
# To get data from external files
import numpy as np
import pandas as pd
from io import BytesIO

# To plot the data
import matplotlib.pyplot as plt
import seaborn as sns
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
from bokeh.models import HoverTool

# To use basic math properties
import math

# Importing and Filtering Data

In [5]:
default_data = pd.read_csv('Rainfall Datasets/default data.csv')
default_data.loc[default_data['RAINFALL']<0,'RAINFALL'] = 0

In [6]:
# Converting to Monthly Rainfall, milimeters to centimeters
data_m = default_data.groupby(['YEAR','MONTH'])['RAINFALL'].sum()/10
data_m_unstacked = data_m.unstack(level=0)

# Making data interactive
df_m = data_m_unstacked.interactive

# Plotting the Data
hover_tool = HoverTool(
    tooltips=[('Year','@columns'),
     ('Month','@index'),
      ('Rainfall','@value cm')]
)

plot_m = df_m.hvplot.heatmap(
    x='columns',
    y='index',
    title='Monthly Rainfall in centimeters',
    rot=70,
    xlabel='Year',
    ylabel='Month',
    width=600, height = 400).opts(
        tools=[hover_tool],
        invert_yaxis=True
    )

plot_m

In [7]:
# Showing the Statistics per Year
print(data_m_unstacked.describe())

YEAR        2000       2001        2002      2003       2004       2005  \
count  12.000000  12.000000   12.000000  12.00000  12.000000  12.000000   
mean   28.734167  15.268333   22.018333  17.01500  12.892500  13.256667   
std    26.408977  14.126444   41.090362  17.24522  12.998594  13.315806   
min     2.350000   2.630000    0.750000   0.12000   0.000000   0.000000   
25%     4.930000   5.062500    1.437500   1.06750   3.067500   1.430000   
50%    22.750000  10.505000    5.430000  11.37500  11.310000   7.365000   
75%    45.737500  18.242500   22.872500  34.22500  18.630000  27.080000   
max    89.310000  46.540000  146.880000  42.55000  42.630000  32.880000   

YEAR        2006       2007       2008       2009       2010       2011  \
count  12.000000  12.000000  12.000000  12.000000  12.000000  12.000000   
mean   17.460000  16.479167  18.330833  24.551667  16.351667  27.069167   
std    18.794844  19.077120  17.156368  26.126450  15.878930  26.289573   
min     0.000000   0.120

In [8]:
# Converting to Annual Data, milimeters to meters
data_a = default_data.groupby('YEAR')['RAINFALL'].sum()/1000
rf_a = data_a.values
color_a = [{r==rf_a.min():'#85C1E9',
            rf_a.min()<r<rf_a.max():'#3498DB',
            r==rf_a.max():'#1F618D'}[True] for r in rf_a]

data_a = pd.DataFrame({'RAINFALL':data_a.values},index=data_a.index)
data_a['color'] = color_a
data_a['MEDIAN_DIFF'] = data_a['RAINFALL'] - \
  data_a['RAINFALL'].median()
data_a['color_diff'] = [{r<0:'red',0<=r:'green'}[True] for r
                        in data_a['MEDIAN_DIFF'].values]
data_a['MED_DIFF_C'] = data_a['MEDIAN_DIFF'].cumsum()
df_a = data_a.interactive

In [9]:
plot1_a = df_a.hvplot.bar(
    x='YEAR',
    y='RAINFALL',
    xlabel='Year',
    ylabel='',
    color='#3498DB',
    title='Rainfall per Year',
    width=700, height=500)

plot1_a

In [10]:
hover_tool = HoverTool(
    tooltips=[('Year','@YEAR'),
     ('Rainfall','@RAINFALL m' )]
)

plot2_a = df_a.hvplot.bar(
    x='YEAR',
    y='RAINFALL',
    xlabel='Year',
    ylabel='',
    color='color',
    title='Rainfall per Year',
    width=700, height=500).opts(
        tools=[hover_tool]
    )

hline = hv.HLine(data_a['RAINFALL'].median()).opts(
        color='black',
        line_dash='dashed')
plot2_a * hline

In [11]:
hover_tool = HoverTool(
    tooltips=[('Year','@YEAR'),
     ('Difference to Median','@MEDIAN_DIFF{0.0000} m' )]
)

plot3_a = df_a.hvplot.bar(
    x='YEAR',
    y='MEDIAN_DIFF',
    xlabel='Year',
    ylabel='',
    color='color_diff',
    title='Difference to Median Rainfall',
    width=700, height=500).opts(
        tools=[hover_tool]
    )

hline = hv.HLine(0).opts(
        color='black')
plot3_a

In [12]:
# Arranging the Rainfall Values in Increasing Order
rainfall_values = pd.DataFrame({'RAINFALL':default_data['RAINFALL'].values})
rainfall_values = rainfall_values.sort_values(by='RAINFALL',ignore_index=True)

# Getting the Plotting Position
rainfall_values['POSITION'] = list(range(1,rainfall_values.count().iloc[0]+1,1))\
/(rainfall_values.count().iloc[0]+1)

df = rainfall_values
idf = df.interactive
rf_stats = idf.hvplot.line(
    x='RAINFALL',
    y='POSITION',
    width=700,height=700
).opts(tools=['crosshair','hover'])

rf_stats

# Dashboard Components

In [ ]:
file_input = pn.widgets.FileInput(accept='.csv',multiple=False)

In [ ]:
submit = pn.widgets.Button(name='Submit', button_type='primary')

# Dashboard Interactivity

In [18]:
def submit_file(clicked):
    if clicked:
        return pd.read_csv(BytesIO(file_input.value))
    return  pd.read_csv('Rainfall Datasets/default data.csv')
    
uploaded_data = pn.bind(submit_file,submit)

In [19]:
def clean_data(dataset):
    dataset.loc[dataset['RAINFALL']<0,'RAINFALL'] = 0
    return dataset
    
cleaned_data = pn.bind(clean_data, uploaded_data)

In [20]:
def convert_to_year(dataset):
    data_a = dataset.groupby('YEAR')['RAINFALL'].sum()/1000
    
    rf_a = data_a.values
    color_a = [{r==rf_a.min():'#85C1E9',
                rf_a.min()<r<rf_a.max():'#3498DB',
                r==rf_a.max():'#1F618D'}[True] for r in rf_a]
    
    data_a = pd.DataFrame({'RAINFALL':data_a.values},index=data_a.index)
    data_a['color'] = color_a
    
    data_a['MEDIAN_DIFF'] = data_a['RAINFALL'] - \
      data_a['RAINFALL'].median()
    data_a['color_diff'] = [{r<0:'red',0<=r:'green'}[True] for r
                            in data_a['MEDIAN_DIFF'].values]
    data_a['MED_DIFF_C'] = data_a['MEDIAN_DIFF'].cumsum()
    return data_a
    
yearly_data = pn.bind(convert_to_year, cleaned_data)

In [21]:
table = pn.widgets.StaticText(value=uploaded_data)
table2 = pn.widgets.StaticText(value=cleaned_data)

# Creating the Dashboard

In [23]:
template = pn.template.FastListTemplate(
     title='Rainfall Analysis',
     sidebar=[
         pn.pane.Markdown("## Upload Data:"),
         file_input,
         submit,
     ],
     main=[
         table,
         pn.widgets.StaticText(value="test"),
         table2,
         pn.pane.Markdown("# Yearly Rainfall Trend"),
         
         pn.pane.Markdown("# Monthly Rainfall Trend"),
         
         pn.pane.Markdown("# Daily Rainfall Percentile"),

          ]
)
template.servable();